In [145]:
import numpy as np
from random import choices , randint, randrange , random
from typing import Callable 
from functools import partial

### initial paramenters

In [163]:
things = [{"name" : "laptop" , "value" : 500,"weight" : 2200},
         {"name" : "headphone" , "value" : 150,"weight" : 160},
         {"name" : "coffe mug" , "value" : 60,"weight" : 350},
         {"name" : "notepad" , "value" : 40,"weight" : 333},
         {"name" : "water bottle" , "value" : 38,"weight" : 192}]

chrom = list[int]
pop = list[chrom]

weightLim = 3000
nPop = 10
cLen = len(things)
fitLimit = 3000 # satisfied limit for us
NGen = 100 # number of generations

fitFunc = Callable[[chrom] , int]
popFun = Callable[[] , pop]
selFunc = Callable[[ pop , fitFunc] , tuple[chrom , chrom]]
crossFun = Callable[[chrom , chrom] , tuple[chrom , chrom]]
mutFunc = Callable[[chrom] , chrom]



### Generating Population

In [164]:
def get_chromosome(n):
    # n --> length of chromosome
    chromosome = choices([0,1] , k = n)
    return chromosome


In [165]:
#test
get_chromosome(10)

[0, 0, 0, 1, 1, 0, 1, 1, 1, 0]

In [166]:
def get_population(nPop , lenChrom):
    """
        nPop --> size of population
        lenChrom --> length of chromosome 
    
    """
    
    pop = [get_chromosome(lenChrom) for i in range(nPop)]
    return pop
    

In [167]:
population = get_population(10 , 4)
population

[[1, 0, 1, 1],
 [0, 1, 0, 0],
 [1, 0, 1, 1],
 [0, 1, 0, 1],
 [1, 0, 0, 1],
 [1, 1, 0, 0],
 [1, 1, 1, 0],
 [0, 0, 1, 0],
 [1, 1, 1, 0],
 [0, 1, 0, 0]]

### Generating Fitness Function

In [168]:
def get_fitness(chrom , things, limit):
    """
        chrom --> chromosome
        things --> thing name and thing weight
        limit --> weight limit
    
    """
    if len(chrom) != len(things):
        raise ValueError("chromosome and things must have the same size")
    
    value = 0
    weight = 0
    
    for i , thing in enumerate(things):
        if chrom[i] == 1:
            value += thing["value"]
            weight += thing["weight"]
    
    if(weight > limit):
        return 0
    
    
    return value
            
    

In [169]:
#test

fitness = get_fitness(get_chromosome(5) , things , limit = 5000)
fitness

288

### Selecting parents

In [170]:
def select_two_parents(pop , fitFun):
    """
        pop --> population
        fitFun --> fitness function
    
    """
    
    twoParents = choices(
        population = pop,
        weights = [fitFun(chrom) for chrom in pop],
        k = 2
        
    )
    
    return twoParents
    

### Generate Crossovver

In [171]:
def get_crossover(first , second):
    """
        first --> first parent
        second --> second parent
    
    """
    
    if len(first) != len(second):
        raise ValueError("two parents must have the same size")
    
    l = len(first)
    randNum = randint(1 , l-1)
    chrom1 = first[:randNum] + second[randNum : ]
    chrom2 = second[:randNum] + first[randNum:]
    return chrom1 , chrom2

In [172]:
# test
chrom1 , chrom2 = get_crossover("11111" , "00000")
chrom1 , chrom2

('11000', '00111')

### Generate Mutation

In [173]:
def get_mutation(chrom , n , prob):
    """
        chrom --> chromosome
        n --> number of iterations
        prob --> probaility
    """
    
    while(n > 0):
        n -= 1
        idx = randrange(len(chrom))
        chrom[idx] = chrom[idx] if random() > prob else abs(chrom[idx] - 1)
    return chrom

In [174]:
# test
get_mutation([1,0,1,0,1,0] , 5 , 0.5)

[0, 0, 1, 1, 1, 0]

### Build Evolution Model

In [175]:
def genetic_model(popFunc , fitFunc , fitLimit , selFunc , crossFunc,
                 mutFunc , gen):
    
    """
        popFunc : population function
        fitFunc : fitness function
        fitlimit : fitness limit
        selFunc : selection function
        crossFunc : crossover function
        mutFunc : mutation function
        gen : number of generations
    
    """
    
    # generate first population population
    pop = popFunc()
    
    for i in range(gen):
        
        # first we calcuate fitness for each chromosome
        # get sorted population based on fitness
        pop = sorted(
            pop , 
            key = lambda chrom : fitFunc(chrom),
            reverse = True
        )
    
        if(fitFunc(pop[0]) > fitLimit):
            break;
    
        nextPop = pop[0:2]
        
        for j in range((len(pop) // 2)-1):
            chrom1 , chrom2 = selFunc(pop , fitFunc)
            chrom1 , chrom2 = crossFunc(chrom1 , chrom2)
            chrom1 = mutFunc(chrom1)
            chrom2 = mutFunc(chrom2)
            nextPop += [chrom1 , chrom2]
        
        pop = nextPop
    
    pop = sorted(
            pop , 
            key = lambda chrom : fitFunc(chrom),
            reverse = True
    )
    
    return pop, i
        
    

### Run Model

In [176]:
pop , iters = genetic_model(
    popFunc = partial(
        get_population , nPop = nPop , lenChrom = cLen
    ),
    fitFunc = partial(
        get_fitness , things = things, limit = weightLim 
    ),
    fitLimit = 3000,
    
    selFunc = partial(
        select_two_parents
    ),
    crossFunc = partial(
        get_crossover
        
    ),
    mutFunc = partial(
        get_mutation , n = 1 , prob = 0.5
    ),
    gen = NGen
    
)

### decorate result

In [177]:
def transform_Chrom(chrom , things):
    res = []
    for i , thing in enumerate(things):
        if chrom[i] == 1:
            res += [thing["name"]]
    return res

In [179]:
print(f"number of iterations: {iters}")

number of iterations: 99


In [180]:
print(transform_Chrom(pop[0] , things))

['laptop', 'headphone', 'coffe mug', 'water bottle']
